# MVP Award

In [47]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

awards_data = pd.read_csv("../../initial_data/awards_players.csv")
teams_data = pd.read_csv("../../initial_data/teams.csv")
players_teams_clean = pd.read_csv("../awards_data/players_teams_clean.csv")

In [48]:
# Check the column names in teams_data
print("Teams data columns:")
print(teams_data.columns.tolist())
print("\nPlayers teams clean columns:")
print(players_teams_clean.columns.tolist())
print("\nTeams data sample:")
print(teams_data.head())

Teams data columns:
['year', 'lgID', 'tmID', 'franchID', 'confID', 'divID', 'rank', 'playoff', 'seeded', 'firstRound', 'semis', 'finals', 'name', 'o_fgm', 'o_fga', 'o_ftm', 'o_fta', 'o_3pm', 'o_3pa', 'o_oreb', 'o_dreb', 'o_reb', 'o_asts', 'o_pf', 'o_stl', 'o_to', 'o_blk', 'o_pts', 'd_fgm', 'd_fga', 'd_ftm', 'd_fta', 'd_3pm', 'd_3pa', 'd_oreb', 'd_dreb', 'd_reb', 'd_asts', 'd_pf', 'd_stl', 'd_to', 'd_blk', 'd_pts', 'tmORB', 'tmDRB', 'tmTRB', 'opptmORB', 'opptmDRB', 'opptmTRB', 'won', 'lost', 'GP', 'homeW', 'homeL', 'awayW', 'awayL', 'confW', 'confL', 'min', 'attend', 'arena']

Players teams clean columns:
['playerID', 'year', 'team', 'minutes', 'games_played', 'total_points', 'total_rebounds', 'total_assists', 'points_per_min', 'assists_per_min', 'rebounds_per_min', 'steals_per_min', 'blocks_per_min', 'turnovers_per_min', 'FG%', 'FT%', 'Three%', 'Three Rate']

Teams data sample:
   year  lgID tmID franchID confID  divID  rank playoff  seeded firstRound  \
0     9  WNBA  ATL      ATL    

## Create MVP Data

In [ ]:
# Join dataset players_teams_clean data with teams_data (team_wins, team_losses, rank, playoff_made)
# Note: players_teams_clean has 'team' column, teams_data has 'tmID' column
mvp_data = players_teams_clean.merge(
    teams_data[['year', 'tmID', 'won', 'lost', 'rank', 'playoff', 'confID']], 
    left_on=['year', 'team'], 
    right_on=['year', 'tmID'], 
    how='left'
)

# Rename columns for clarity
mvp_data.rename(columns={
    'won': 'team_wins', 
    'lost': 'team_losses', 
    'rank': 'conference_rank', 
    'playoff': 'playoff_made'
}, inplace=True)

# Drop the duplicate tmID column (we already have 'team')
mvp_data.drop(columns=['tmID'], inplace=True)

# Change Playoff from N / Y to 0 / 1
mvp_data['playoff_made'] = mvp_data['playoff_made'].map({'Y': 1, 'N': 0})
# Change ConfID to numeric  EA / WE to 0 / 1
mvp_data['confID'] = mvp_data['confID'].astype('category').cat.codes

print(f"MVP data shape: {mvp_data.shape}")
print(f"\nColumns: {mvp_data.columns.tolist()}")
print(f"\nSample data:")
print(mvp_data.head())

# Save csv (relative path from current location)
mvp_data.to_csv("mvp_data.csv", index=False)
print(f"\n✅ Saved to mvp_data.csv")

MVP data shape: (1876, 23)

Columns: ['playerID', 'year', 'team', 'minutes', 'games_played', 'total_points', 'total_rebounds', 'total_assists', 'points_per_min', 'assists_per_min', 'rebounds_per_min', 'steals_per_min', 'blocks_per_min', 'turnovers_per_min', 'FG%', 'FT%', 'Three%', 'Three Rate', 'team_wins', 'team_losses', 'conference_rank', 'playoff_made', 'confID']

Sample data:
     playerID  year team  minutes  games_played  total_points  total_rebounds  \
0  abrossv01w     2  MIN      846            26           343             174   
1  abrossv01w     3  MIN      805            27           314             146   
2  abrossv01w     4  MIN      792            30           318             141   
3  abrossv01w     5  MIN      462            22           146              74   
4  abrossv01w     6  MIN      777            31           304             107   

   total_assists  points_per_min  assists_per_min  ...  turnovers_per_min  \
0             53        0.405437         0.062648  ..

## Prediction Models

### 1. Logistic Regression

In [51]:
# Prepare MVP prediction dataset
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

# Load data
mvp_data = pd.read_csv("mvp_data.csv")
awards_data = pd.read_csv("../../initial_data/awards_players.csv")

# Step 1: Filter to realistic MVP candidates
# Only players with significant playing time and on winning teams
mvp_candidates = mvp_data[
    (mvp_data['minutes'] >= 800) &  # Significant playing time
    (mvp_data['games_played'] >= 25) &  # Played most of season
    (mvp_data['team_wins'] / (mvp_data['team_wins'] + mvp_data['team_losses']) >= 0.500)  # Winning team
].copy()

# Step 2: Add MVP winner target variable
mvp_winners = awards_data[awards_data['award'] == 'Most Valuable Player'][['year', 'playerID']]
mvp_candidates['MVP_winner'] = mvp_candidates.apply(
    lambda row: 1 if ((mvp_winners['year'] == row['year']) & 
                      (mvp_winners['playerID'] == row['playerID'])).any() else 0,
    axis=1
)

print(f"Total MVP candidates: {len(mvp_candidates)}")
print(f"MVP winners in dataset: {mvp_candidates['MVP_winner'].sum()}")
print(f"Years available: {sorted(mvp_candidates['year'].unique())}")
print(f"\nClass distribution:")
print(mvp_candidates['MVP_winner'].value_counts())
print(f"\nSample data:")
print(mvp_candidates[['playerID', 'year', 'team_wins', 'conference_rank', 'MVP_winner']].head(10))

Total MVP candidates: 310
MVP winners in dataset: 10
Years available: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

Class distribution:
MVP_winner
0    300
1     10
Name: count, dtype: int64

Sample data:
      playerID  year  team_wins  conference_rank  MVP_winner
32  andrame01w     2         22                1           0
37  arcaija01w     1         27                2           0
38  arcaija01w     2         19                4           0
39  arcaija01w     3         24                2           0
40  arcaija01w     4         20                2           0
41  arcaija01w     6         19                3           0
59   azzije01w     2         19                3           0
60   azzije01w     3         20                3           0
74  baranel01w     2         20                3           0
76  baranel01w     5         18                2           0


In [52]:
# Define feature columns for the model
feature_columns = [
    'playerID', 'year', 'team','minutes','games_played','total_points','total_rebounds','total_assists','points_per_min','assists_per_min',
    'rebounds_per_min','steals_per_min','blocks_per_min','turnovers_per_min','FG%','FT%','Three%','Three Rate','team_wins','team_losses',
    'conference_rank','playoff_made','confID']

print("Features for MVP prediction:")
for i, col in enumerate(feature_columns, 1):
    print(f"{i}. {col}")

Features for MVP prediction:
1. playerID
2. year
3. team
4. minutes
5. games_played
6. total_points
7. total_rebounds
8. total_assists
9. points_per_min
10. assists_per_min
11. rebounds_per_min
12. steals_per_min
13. blocks_per_min
14. turnovers_per_min
15. FG%
16. FT%
17. Three%
18. Three Rate
19. team_wins
20. team_losses
21. conference_rank
22. playoff_made
23. confID


In [53]:
# MVP Prediction Pipeline - Configure Training and Test Years

# Available years
available_years = sorted(mvp_candidates['year'].unique())
print(f"Available years: {available_years}")

# Configure training and test years
# Modify these lists based on your needs
train_years = [1, 2, 3, 4, 5, 6, 7, 8]  # Years to train the model
test_years = [9, 10]  # Years to test the model

print(f"\n📚 Training years: {train_years}")
print(f"🧪 Test years: {test_years}")

# Verify years exist in dataset
missing_train = [y for y in train_years if y not in available_years]
missing_test = [y for y in test_years if y not in available_years]

if missing_train:
    print(f"⚠️  Warning: Training years {missing_train} not found in dataset")
if missing_test:
    print(f"⚠️  Warning: Test years {missing_test} not found in dataset")
    
# Prepare train/test splits
X_train = mvp_candidates[mvp_candidates['year'].isin(train_years)][feature_columns]
y_train = mvp_candidates[mvp_candidates['year'].isin(train_years)]['MVP_winner']
X_test = mvp_candidates[mvp_candidates['year'].isin(test_years)][feature_columns]
y_test = mvp_candidates[mvp_candidates['year'].isin(test_years)]['MVP_winner']

# Keep full records for detailed analysis
train_data = mvp_candidates[mvp_candidates['year'].isin(train_years)].copy()
test_data = mvp_candidates[mvp_candidates['year'].isin(test_years)].copy()

print(f"\n📊 Dataset Split:")
print(f"Training set: {len(X_train)} candidates, {y_train.sum()} MVP winners")
print(f"Test set: {len(X_test)} candidates, {y_test.sum()} MVP winners")

Available years: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

📚 Training years: [1, 2, 3, 4, 5, 6, 7, 8]
🧪 Test years: [9, 10]

📊 Dataset Split:
Training set: 255 candidates, 8 MVP winners
Test set: 55 candidates, 2 MVP winners


In [54]:
# Train the MVP Prediction Model
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

# Scale features (important for logistic regression)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train Logistic Regression model with balanced class weights
# This handles the imbalanced dataset (few MVP winners vs many candidates)
model = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42)
model.fit(X_train_scaled, y_train)

# Make predictions
y_pred = model.predict(X_test_scaled)
y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]  # Probability of being MVP

# Store predictions in test data
test_data['mvp_probability'] = y_pred_proba
test_data['mvp_prediction'] = y_pred

print("✅ Model training complete!")
print(f"Training accuracy: {model.score(X_train_scaled, y_train):.3f}")
print(f"Test accuracy: {model.score(X_test_scaled, y_test):.3f}")

ValueError: could not convert string to float: 'andrame01w'

In [ ]:
# Model Evaluation
from sklearn.metrics import classification_report, confusion_matrix

print("=" * 60)
print("MODEL EVALUATION RESULTS")
print("=" * 60)

# Classification report
print("\n📊 Classification Report:")
print(classification_report(y_test, y_pred, target_names=['Not MVP', 'MVP']))

# Confusion matrix
print("\n🔍 Confusion Matrix:")
cm = confusion_matrix(y_test, y_pred)
print(f"True Negatives: {cm[0, 0]} | False Positives: {cm[0, 1]}")
print(f"False Negatives: {cm[1, 0]} | True Positives: {cm[1, 1]}")

# Feature importance
print("\n⭐ Top 10 Most Important Features:")
feature_importance = pd.DataFrame({
    'feature': feature_columns,
    'coefficient': model.coef_[0]
}).sort_values('coefficient', ascending=False)

print("\nPositive impact (increases MVP probability):")
print(feature_importance.head(5).to_string(index=False))
print("\nNegative impact (decreases MVP probability):")
print(feature_importance.tail(5).to_string(index=False))

In [ ]:
# Per-Year MVP Predictions with Rankings

print("=" * 60)
print("MVP PREDICTIONS BY YEAR")
print("=" * 60)

for year in sorted(test_data['year'].unique()):
    year_data = test_data[test_data['year'] == year].copy()
    year_data = year_data.sort_values('mvp_probability', ascending=False)
    
    print(f"\n{'='*60}")
    print(f"🏀 Year {year} - Top 10 MVP Candidates")
    print(f"{'='*60}")
    
    # Get actual MVP if exists
    actual_mvp = year_data[year_data['MVP_winner'] == 1]
    if len(actual_mvp) > 0:
        actual_mvp_name = actual_mvp.iloc[0]['playerID']
        print(f"🏆 Actual MVP: {actual_mvp_name}")
    else:
        print("🏆 Actual MVP: Data not available")
    
    # Show top 10 predictions
    top_10 = year_data.head(10)
    
    print(f"\n{'Rank':<6}{'Player':<25}{'Team':<6}{'MVP Prob':<12}{'Predicted':<12}{'PPG':<8}{'Wins':<6}")
    print("-" * 85)
    
    for idx, (_, row) in enumerate(top_10.iterrows(), 1):
        is_mvp = "✅ MVP" if row['MVP_winner'] == 1 else ""
        predicted = "🎯 PRED" if row['mvp_prediction'] == 1 else ""
        
        print(f"{idx:<6}{row['playerID']:<25}{row['team']:<6}"
              f"{row['mvp_probability']:.4f}{'':6}{predicted:<12}"
              f"{row['points_per_game']:.1f}{'':6}{int(row['team_wins']):<6}{is_mvp}")
    
    # Prediction summary
    predicted_mvp = year_data[year_data['mvp_prediction'] == 1]
    if len(predicted_mvp) > 0:
        pred_name = predicted_mvp.iloc[0]['playerID']
        pred_prob = predicted_mvp.iloc[0]['mvp_probability']
        print(f"\n🎯 Model's MVP Pick: {pred_name} (probability: {pred_prob:.4f})")
    else:
        top_candidate = year_data.iloc[0]
        print(f"\n🎯 Model's Top Candidate: {top_candidate['playerID']} "
              f"(probability: {top_candidate['mvp_probability']:.4f})")

print(f"\n{'='*60}")
print("Analysis Complete!")
print(f"{'='*60}")